# TCC - Uso de _transformers_ na previsão da evasão de IES públicas no Brasil
## Preparação dos dados

## Imports

In [1]:
import time
import warnings
import pandas as pd

from pathlib import Path

warnings.filterwarnings('ignore') 

## Importação dos dados dos arquivos CSV

In [2]:
data_path = Path('data')

### Dados de trajetória

In [3]:
filepath = data_path / 'indicadores_trajetoria_educacao_superior_2014_2023.xlsx'

df_trajetoria = pd.read_excel(filepath, header=8)
df_trajetoria = df_trajetoria[df_trajetoria['NO_CURSO'] == 'ENGENHARIA ELÉTRICA']

df_trajetoria.head(3)

,CO_IES,NO_IES,TP_CATEGORIA_ADMINISTRATIVA,TP_ORGANIZACAO_ACADEMICA,CO_CURSO,NO_CURSO,CO_REGIAO,CO_UF,CO_MUNICIPIO,TP_GRAU_ACADEMICO,...,QT_INGRESSANTE,QT_PERMANENCIA,QT_CONCLUINTE,QT_DESISTENCIA,QT_FALECIDO,TAP,TCA,TDA,TCAN,TADA
110,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1.0,1.0,16.0,ENGENHARIA ELÉTRICA,5.0,51.0,5103403.0,1.0,...,77.0,75.0,0.0,2.0,0.0,97.402597,0.0,2.597403,0.0,2.597403
111,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1.0,1.0,16.0,ENGENHARIA ELÉTRICA,5.0,51.0,5103403.0,1.0,...,77.0,71.0,0.0,4.0,0.0,92.207792,0.0,7.792208,0.0,5.194805
112,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1.0,1.0,16.0,ENGENHARIA ELÉTRICA,5.0,51.0,5103403.0,1.0,...,77.0,68.0,0.0,3.0,0.0,88.311688,0.0,11.688312,0.0,3.896104


In [4]:
df_trajetoria = df_trajetoria[['CO_IES', 'NU_ANO_REFERENCIA', 'QT_DESISTENCIA']].groupby(['CO_IES', 'NU_ANO_REFERENCIA']).sum().reset_index()
df_trajetoria = df_trajetoria.astype(int)
df_trajetoria.head(10)

,CO_IES,NU_ANO_REFERENCIA,QT_DESISTENCIA
0,1,2014,2
1,1,2015,4
2,1,2016,3
3,1,2017,6
4,1,2018,8
5,1,2019,6
6,1,2020,3
7,1,2021,0
8,1,2022,1
9,1,2023,7


### Dados das instituições

In [5]:
IMPORTANT_COLUMNS = [
    "NU_ANO_CENSO",
    "NO_REGIAO",
    "CO_IES",
    "QT_ING",
    "QT_ING_FEM",
    "QT_ING_MASC",
    "QT_ING_BRANCA",
    "QT_ING_PRETA",
    "QT_ING_PARDA",
    "QT_ING_AMARELA",
    "QT_ING_INDIGENA",
    "QT_MAT",
    "QT_MAT_FEM",
    "QT_MAT_MASC",
    "QT_MAT_BRANCA",
    "QT_MAT_PRETA",
    "QT_MAT_PARDA",
    "QT_MAT_AMARELA",
    "QT_MAT_INDIGENA",
    "QT_ALUNO_DEFICIENTE",
    "QT_ING_RESERVA_VAGA",
    "QT_MAT_RESERVA_VAGA",
    "QT_ING_PROCESCPUBLICA",
    "QT_ING_PROCESCPRIVADA",
    "QT_MAT_PROCESCPUBLICA",
    "QT_MAT_PROCESCPRIVADA",
    "QT_APOIO_SOCIAL"
]

In [6]:
def prepare_data(year:          int, 
                 df:            pd.DataFrame, 
                 df_trajetoria: pd.DataFrame = df_trajetoria, 
                 cols:          list[str]    = IMPORTANT_COLUMNS) -> pd.DataFrame:

    # ---| Pt. 1: Filtra os cursos de Engenharia Elétrica da Rede Pública Federal
    df['NO_CURSO'] = df['NO_CURSO'].apply(lambda x: x.upper())

    filt1 = df['TP_REDE']  == 1 # tipo 1: rede pública
    filt2 = df['NO_CURSO'] == 'ENGENHARIA ELÉTRICA'
    df    = df[filt1&filt2]

    # ---| Pt. 2: Separa-se as colunas importantes
    df = df[cols]

    # ---| Pt. 3: Obtem-se a quantidade total de alunos
    to_get_total = ['QT_ING', 'QT_MAT']
    df['QT_TOT'] = df[to_get_total].sum(axis=1)
    df.drop(columns=to_get_total, inplace=True)

    # ---| Pt. 4: Agrupa-se as colunas semelhantes
    grouped_cols = dict()

    for col in cols:
        if col.startswith('QT'):
            broke_str = col.split('_')
            broke_str.pop(1)
            group = '_'.join(broke_str)

            if group not in grouped_cols.keys():
                grouped_cols[group] = [col]
            else:
                grouped_cols[group].append(col)

    for key in ['QT', 'QT_SOCIAL', 'QT_DEFICIENTE']:
        grouped_cols.pop(key)

    # ---| Pt. 5: Obtem-se a quantidade total de alunos agrupados
    for new_col, old_cols in grouped_cols.items():
        df[new_col] = df[old_cols].sum(axis=1)
        df.drop(columns=old_cols, inplace=True)

    # ---| Pt. 6: Agrupa-se por instituição
    df = df.groupby(['CO_IES', 'NU_ANO_CENSO', 'NO_REGIAO']).sum()
    df.reset_index(inplace=True)

    # ---| Pt. 7: Adiciona-se o número de evadidos
    df_trajetoria = df_trajetoria[df_trajetoria['NU_ANO_REFERENCIA'] == year]
    df_trajetoria.drop('NU_ANO_REFERENCIA', axis=1, inplace=True)
    df = df.merge(df_trajetoria, on=['CO_IES'])
    
    return df.reset_index(drop=True)

In [7]:
all_data = dict()
total_data = 0

for index, filepath in enumerate(data_path.glob("*.CSV")):
    if 'CURSOS' in filepath.name:
        if index: print()
        try:
            start = time.time()
            print(f'> Abrindo {filepath.name}...')
            year = int(filepath.name.rsplit('_')[-1][:4])
            df   = pd.read_csv(filepath, encoding="ISO-8859-1", sep=';')
            total_data += df.shape[0]

            all_data[year] = prepare_data(year, df)
            
            print(f'> Arquivo aberto e tratado com sucesso! Tempo passado: {time.time()-start:.2f}s')
            display(all_data[year].head(3))
        except Exception as e:
            print(f'> {e}')
    else:
        break

> Abrindo MICRODADOS_CADASTRO_CURSOS_2014.CSV...
> Arquivo aberto e tratado com sucesso! Tempo passado: 2.82s


,CO_IES,NU_ANO_CENSO,NO_REGIAO,QT_ALUNO_DEFICIENTE,QT_APOIO_SOCIAL,QT_TOT,QT_FEM,QT_MASC,QT_BRANCA,QT_PRETA,QT_PARDA,QT_AMARELA,QT_INDIGENA,QT_RESERVA_VAGA,QT_PROCESCPUBLICA,QT_PROCESCPRIVADA,QT_DESISTENCIA
0,1,2014,Centro-Oeste,0.0,32.0,435.0,72.0,363.0,195.0,55.0,155.0,18.0,2.0,123.0,170.0,210.0,2
1,2,2014,Centro-Oeste,2.0,21.0,534.0,100.0,434.0,293.0,50.0,138.0,13.0,4.0,99.0,133.0,401.0,5
2,3,2014,Nordeste,2.0,15.0,326.0,57.0,269.0,53.0,38.0,189.0,6.0,1.0,44.0,157.0,166.0,2



> Abrindo MICRODADOS_CADASTRO_CURSOS_2015.CSV...
> Arquivo aberto e tratado com sucesso! Tempo passado: 1.71s


,CO_IES,NU_ANO_CENSO,NO_REGIAO,QT_ALUNO_DEFICIENTE,QT_APOIO_SOCIAL,QT_TOT,QT_FEM,QT_MASC,QT_BRANCA,QT_PRETA,QT_PARDA,QT_AMARELA,QT_INDIGENA,QT_RESERVA_VAGA,QT_PROCESCPUBLICA,QT_PROCESCPRIVADA,QT_DESISTENCIA
0,1,2015,Centro-Oeste,0.0,29.0,430.0,77.0,353.0,191.0,44.0,180.0,6.0,2.0,151.0,201.0,199.0,4
1,2,2015,Centro-Oeste,1.0,31.0,567.0,110.0,457.0,298.0,49.0,154.0,16.0,3.0,129.0,155.0,412.0,6
2,3,2015,Nordeste,3.0,14.0,313.0,60.0,253.0,80.0,47.0,175.0,10.0,1.0,65.0,145.0,167.0,17



> Abrindo MICRODADOS_CADASTRO_CURSOS_2016.CSV...
> Arquivo aberto e tratado com sucesso! Tempo passado: 2.63s


...


> Abrindo MICRODADOS_CADASTRO_CURSOS_2021.CSV...
> Arquivo aberto e tratado com sucesso! Tempo passado: 39.89s


,CO_IES,NU_ANO_CENSO,NO_REGIAO,QT_ALUNO_DEFICIENTE,QT_APOIO_SOCIAL,QT_TOT,QT_FEM,QT_MASC,QT_BRANCA,QT_PRETA,QT_PARDA,QT_AMARELA,QT_INDIGENA,QT_RESERVA_VAGA,QT_PROCESCPUBLICA,QT_PROCESCPRIVADA,QT_DESISTENCIA
0,1,2021,Centro-Oeste,9,3,389,70,319,162,38,174,9,0,163,223,166,0
1,2,2021,Centro-Oeste,4,34,477,65,412,151,34,253,5,0,199,249,228,0
2,3,2021,Nordeste,2,39,300,31,269,78,38,178,5,1,130,159,141,2



> Abrindo MICRODADOS_CADASTRO_CURSOS_2022.CSV...
> Arquivo aberto e tratado com sucesso! Tempo passado: 38.34s


,CO_IES,NU_ANO_CENSO,NO_REGIAO,QT_ALUNO_DEFICIENTE,QT_APOIO_SOCIAL,QT_TOT,QT_FEM,QT_MASC,QT_BRANCA,QT_PRETA,QT_PARDA,QT_AMARELA,QT_INDIGENA,QT_RESERVA_VAGA,QT_PROCESCPUBLICA,QT_PROCESCPRIVADA,QT_DESISTENCIA
0,1,2022,Centro-Oeste,9,21,372,84,288,137,43,179,9,0,164,223,149,1
1,2,2022,Centro-Oeste,4,75,512,74,438,196,36,274,6,0,201,339,172,4
2,3,2022,Nordeste,2,29,313,43,270,91,48,168,5,1,149,182,131,0



> Abrindo MICRODADOS_CADASTRO_CURSOS_2023.CSV...
> Unable to allocate 958. MiB for an array with shape (187, 671610) and data type int64


### Junção dos dados

In [8]:
df_to_train = pd.concat(all_data.values(), ignore_index=True)
df_to_train.head(3)

,CO_IES,NU_ANO_CENSO,NO_REGIAO,QT_ALUNO_DEFICIENTE,QT_APOIO_SOCIAL,QT_TOT,QT_FEM,QT_MASC,QT_BRANCA,QT_PRETA,QT_PARDA,QT_AMARELA,QT_INDIGENA,QT_RESERVA_VAGA,QT_PROCESCPUBLICA,QT_PROCESCPRIVADA,QT_DESISTENCIA
0,1,2014,Centro-Oeste,0.0,32.0,435.0,72.0,363.0,195.0,55.0,155.0,18.0,2.0,123.0,170.0,210.0,2
1,2,2014,Centro-Oeste,2.0,21.0,534.0,100.0,434.0,293.0,50.0,138.0,13.0,4.0,99.0,133.0,401.0,5
2,3,2014,Nordeste,2.0,15.0,326.0,57.0,269.0,53.0,38.0,189.0,6.0,1.0,44.0,157.0,166.0,2


In [9]:
df_to_train.shape

(538, 17)

In [10]:
filename = 'treated_db.csv'
df_to_train.to_csv(data_path / filename, index=False)

In [11]:
total_data

1912777